In [1]:
import numpy as np
import matplotlib.pyplot as plt
import stim
from lib.stabilizer import measurement_gadgets, StabilizerCode, stabilizer_circuits
from lib.color_compass import Lattice2D, compass_to_surface
from lib.decoder import checkmatrix,pL_from_checkmatrix
from lib.stim2pymatching import estimate_pL_noisy_graph
import stimcirq
from typing import *
from cirq.contrib.svg import SVGCircuit
import pymatching

The basic learning loop is as follows 

0) Fix a hidden Pauli noise model for the gates and measurements
1) Pick a compass code and get its stabilizers and logical operators
2) Construct a noisy stim circuit for the code under the noise model 
3) Simulate and get logical error rate
4) Go to 1) and repeat

In [2]:
# Construction of a Pauli noise model

class PauliNoiseModel():
    """
    Constructs noisy Stim circuits from 2D Compass Code stabilizers and logical observables

    A noise model is defined as mapping a perfect operation to an imperfect operation
    """
    def __init__(self, one_qb_gate_rates : List[float] = [0] * 3, two_qb_gate_rates : List[float] = [0] * 15, meas_error_rate : float = 0):
        self.one_qb_gate_rates = one_qb_gate_rates
        assert(len(one_qb_gate_rates) == 3)
        self.two_qb_gate_rates = two_qb_gate_rates
        assert(len(two_qb_gate_rates) == 15)
        self.meas_error_rate = meas_error_rate

    def one_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a single qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_1({},{},{})'.format(*self.one_qb_gate_rates)
        return channel_str 
    
    def two_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a two qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_2({},{},{},{},{},{},{},{},{},{},{},{},{},{},{})'.format(*self.two_qb_gate_rates)
        return channel_str 
    
    def measurement_gadget(self, pauli_observable : str):
        """ 
        Stim gadget to directly measure the specified 'pauli_observable'
        """
        meas_circ = ''
        x_meas_pos = ''
        y_meas_pos = '' 
        z_meas_pos = ''
        pos = {'I' : [], 'X' : [], 'Y' : [], 'Z' : []}
        for i, pauli in enumerate(pauli_observable):
            if pauli == 'X':
                x_meas_pos += f' {i}'
            elif pauli == 'Y':
                y_meas_pos += f' {i}'
            elif pauli == 'Z':
                z_meas_pos += f' {i}'
            else:
                pass 

        meas_circ += f'MX({self.meas_error_rate})' + (x_meas_pos * (len(x_meas_pos) != 0)) + '\n' + f'MY({self.meas_error_rate})' + (y_meas_pos * (len(y_meas_pos) != 0)) + '\n' + f'MZ({self.meas_error_rate})' + (z_meas_pos * (len(z_meas_pos) != 0)) + '\n'
        return stim.Circuit(meas_circ)
    
    def stabilizer_gadget(self, stabilizer_in : str, ancilla_index : int, construction : str = 'cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """
         # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = ''
        if construction == 'cnot':
            for i, pauli in enumerate(stabilizer):
                if pauli == 'Z':
                    # Z-gates are just cnots from data to ancilla
                    noise_string = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string
                elif pauli == 'X':
                    # X-gates are conjugated by hadamards
                    noise_string_1qb = self.one_qb_pauli_noise() + f' {i}\n'
                    noise_string_2qb = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                elif pauli == 'Y':
                    # Y-gates are conjugated by S-gates and hadamards
                    noise_string_1qb = self.one_qb_pauli_noise() + f' {i}\n'
                    noise_string_2qb = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string = f'S_DAG {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'S {i} \n'
                    circ_string += noise_string_1qb 

            # noisy ancilla measurement
            circ_string += f'MR({self.meas_error_rate}) {N+ancilla_index}\n'
        return stim.Circuit(circ_string)


    def stabilizer_gadget_v2(self, stabilizer_in : int):
        """
        Use Stim's built in 'MPP' function
        (IS THIS PREFERRED OVER SPLITTING UP MEASUREMENTS INTO CONSTITUENT PARTS AND APPLYING CIRCUIT-LEVEL NOISE?)
        """
        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = f'MPP({self.meas_error_rate}) '
        for i, pauli in enumerate(stabilizer):
            if (pauli != 'I'):
                circ_string += f'{pauli}{i}*'
        circ_string = circ_string[:-1] + '\n'
        return stim.Circuit(circ_string)

In [3]:
"""pick the compass code"""
dim = 9
lat = Lattice2D(dim, dim)

coloring = np.random.randint(-1, 2, size=(dim-1)**2)
lat.color_lattice(coloring)
print(lat)

000---001---002---003---004---005---006---007---008
 |     |  ▓  |  ░  |  ▓  |     |  ░  |  ▓  |     |
009---010---011---012---013---014---015---016---017
 |  ░  |     |     |  ░  |  ░  |     |  ▓  |  ▓  |
018---019---020---021---022---023---024---025---026
 |     |  ░  |     |  ░  |     |  ▓  |  ░  |  ░  |
027---028---029---030---031---032---033---034---035
 |     |     |     |     |  ▓  |     |  ▓  |  ░  |
036---037---038---039---040---041---042---043---044
 |  ▓  |  ░  |  ░  |  ▓  |  ░  |  ░  |  ▓  |     |
045---046---047---048---049---050---051---052---053
 |  ▓  |  ▓  |  ▓  |     |  ▓  |     |  ▓  |  ░  |
054---055---056---057---058---059---060---061---062
 |     |     |  ▓  |  ░  |  ░  |     |     |  ▓  |
063---064---065---066---067---068---069---070---071
 |  ░  |     |  ▓  |     |  ░  |     |     |  ░  |
072---073---074---075---076---077---078---079---080



In [4]:
"""construct the encoding circuit"""
encoding = StabilizerCode(lat.getS()).encoding_circuit(stim=True)
print(encoding)

H 0 3 4 7 9 10 14 15 18 24 26 27 28 30 31 33 36 37 39 42 45 48 50 52 53 54 56 58 59 60 63 67 69 71
TICK
CX 0 2 9 19 10 11 24 25 27 46 28 29 30 41 42 44 48 49 54 55 63 64
TICK
CX 0 12 9 20 24 35 27 47 28 46 31 41
TICK
CX 0 13 3 12 9 21 26 35 28 47 31 32 36 46 39 41
TICK
CX 0 23 4 13 9 22 10 12 33 35 36 47 37 46
TICK
CX 0 72 4 16 10 13 37 47 45 46
TICK
CX 0 73 4 23 7 16 10 19 37 38 45 47
TICK
CX 0 74 7 8 9 23 10 20 15 16 18 19
TICK
CX 0 51 9 72 10 21 14 23 18 20
TICK
CX 0 76 4 51 9 73 10 22 18 21
TICK
CX 0 61 9 74 18 22
TICK
CX 0 78 4 61 9 51 18 23
TICK
CX 0 1 4 17 9 76 15 51 18 72
TICK
CX 0 75 4 6 7 17 9 61 18 73 27 72
TICK
CX 0 62 4 5 9 78 15 61 18 74 27 57 36 72
TICK
CX 0 77 4 62 9 75 15 17 18 51 27 73 30 57 45 72
TICK
CX 0 79 9 62 18 76 24 51 27 74 36 57 54 72
TICK
CX 0 80 9 77 15 62 18 61 30 51 36 73 39 57 63 72
TICK
CX 9 79 18 78 24 61 30 76 33 51 36 74 45 57
TICK
CX 9 80 18 75 30 61 39 51 45 73 48 57
TICK
CX 18 62 27 75 30 78 33 61 39 76 42 51 45 74 54 73 56 57
TICK
CX 18 77 24 62

In [9]:
"""fix a gate set"""
one_qb_rates = [0.01, 0.01, 0.01]
meas_rates = 0.00
pnm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates)
noiseless_nm = PauliNoiseModel(one_qb_gate_rates=[0.0]*3, meas_error_rate=0.0)

In [6]:
"""make a stabilizer gadgets"""
stab_gadget = pnm.stabilizer_gadget(lat.getS()[0], 0)
print(stab_gadget)

H 0
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 0
CX 0 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 0 81
H 0
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 0
H 1
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 1
CX 1 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 1 81
H 1
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 1
H 2
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 2
CX 2 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 2 81
H 2
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 2
H 9
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 9
CX 9 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 9 81
H 9
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 9
H 10
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 10
CX 10 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 10 81
H 10
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 10
H 11
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 11
CX 11 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 11 81
H 11
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 11
MR(0.01) 81


In [7]:
"""make the observable measurement"""
meas_gadget = pnm.measurement_gadget(lat.Lx)
print(meas_gadget)

MX(0.01) 0 1 2 3 4 5 6 7 8
MY(0.01)
M(0.01)


In [18]:
def compile_compass_circuit(compass_code : Lattice2D, pauli_noise_model : PauliNoiseModel, noiseless_model : PauliNoiseModel, rounds : int):
    """ 
    We compile a compass code lattice into stim circuits with detectors between subsequent stabilizer measurements

    Params:
    * compass_code - Instance of 'Lattice2D' class that defines compass code
    * pauli_noise_model - Instance of 'PauliNoiseModel' that defines Pauli noise model
    * rounds - Number of rounds of stabilizer measurements we look to perform
    """
    
    compass_circuit = stim.Circuit()

    # Perform encoding into logical all-zeros state
    encoding_circ = StabilizerCode(compass_code.getS()).encoding_circuit(stim=True)
    compass_circuit += encoding_circ

    # Add dummy measurements at start of circuit (X stabs)
    num_X_stabs = len(compass_code.getSx())
    num_Z_stabs = len(compass_code.getSz())

    for idx, sx in enumerate(compass_code.getSx()):
        compass_circuit += noiseless_model.stabilizer_gadget(sx, idx)

    for idz, sz in enumerate(compass_code.getSz()):
        compass_circuit += noiseless_model.stabilizer_gadget(sz, idz)

    # Perform n rounds of stabilizer measurements and add detector
    for n in range(rounds):
        if (n > rounds - 1):
            for idx, sx in enumerate(compass_code.getSx()):
                compass_circuit += noiseless_model.stabilizer_gadget(sx, idx)
            for idz, sz in enumerate(compass_code.getSz()):
                compass_circuit += noiseless_model.stabilizer_gadget(sz, idz)
            for idx, sx in enumerate(compass_code.getSx()):
                compass_circuit += stim.Circuit(f"DETECTOR({idx}, 0, {n + 1}) rec[{-1 - idx}] rec[{-1 - num_X_stabs - num_Z_stabs - idx}]")
            for idz, sz in enumerate(compass_code.getSz()):
                compass_circuit += stim.Circuit(f"DETECTOR({idz}, 1, {n + 1}) rec[{-1 - idz}] rec[{-1 - num_Z_stabs - num_X_stabs - idz}]")
        else:
            for idx, sx in enumerate(compass_code.getSx()):
                compass_circuit += pauli_noise_model.stabilizer_gadget(sx, idx)
            for idz, sz in enumerate(compass_code.getSz()):
                compass_circuit += pauli_noise_model.stabilizer_gadget(sz, idz)
            for idx, sx in enumerate(compass_code.getSx()):
                compass_circuit += stim.Circuit(f"DETECTOR({idx}, 0, {n + 1}) rec[{-1 - idx}] rec[{-1 - num_X_stabs - num_Z_stabs - idx}]")
            for idz, sz in enumerate(compass_code.getSz()):
                compass_circuit += stim.Circuit(f"DETECTOR({idz}, 1, {n + 1}) rec[{-1 - idz}] rec[{-1 - num_Z_stabs - num_X_stabs - idz}]")
    
    return compass_circuit 

## Core Learning Loop (w/ PyMatching)

In [19]:
def shot(circuit : stim.Circuit, rounds : int):
    sample = circuit.compile_sampler().sample(rounds)
    sample = sample.astype(int)
    for round in range(rounds):
        print("".join("_1"[e] for e in sample[round]))

def detector_shot(circuit : stim.Circuit, rounds : int):
    sample = circuit.compile_detector_sampler().sample(rounds)
    sample = sample.astype(int)
    for round in range(rounds):
        print("".join("_1"[e] for e in sample[round]))

In [32]:
# Randomly select a compass code
dim = 4
lat = Lattice2D(dim, dim)

coloring = np.random.randint(-1, 2, size=(dim-1)**2)
lat.color_lattice(coloring)
print(lat)

000---001---002---003
 |     |     |  ▓  |
004---005---006---007
 |     |     |  ░  |
008---009---010---011
 |     |  ▓  |  ░  |
012---013---014---015



In [33]:
# Fix a Pauli noise model
one_qb_rates = [0.0] * 3
two_qb_rates = [0.0] * 15
meas_rates = 0.01
pnm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates, two_qb_gate_rates=two_qb_rates)

In [34]:
# Define a noiseless model
one_qb_rates = [0.0] * 3
meas_rates = 0.0
noiseless_nm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates)

In [35]:
# Construct the encoding and measurement circuit for our chosen code
num_stab_meas_rounds = 2
circ = compile_compass_circuit(lat, pnm, noiseless_nm, num_stab_meas_rounds)

In [36]:
print(circ)

H 4 1 3 8 10
TICK
CX 4 5 10 11
TICK
CX 4 6
TICK
CX 4 7
TICK
CX 4 14 1 7
TICK
CX 4 12 1 14 3 7
TICK
CX 4 13 1 0 10 14
TICK
CX 4 15 1 2
TICK
CX 1 12
TICK
CX 1 13 8 12
TICK
CX 1 15 8 13
TICK
CX 8 9 10 15
TICK
H 4
PAULI_CHANNEL_1(0, 0, 0) 4
CX 4 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 4 16
H 4
PAULI_CHANNEL_1(0, 0, 0) 4
H 5
PAULI_CHANNEL_1(0, 0, 0) 5
CX 5 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 5 16
H 5
PAULI_CHANNEL_1(0, 0, 0) 5
H 6
PAULI_CHANNEL_1(0, 0, 0) 6
CX 6 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 6 16
H 6
PAULI_CHANNEL_1(0, 0, 0) 6
H 7
PAULI_CHANNEL_1(0, 0, 0) 7
CX 7 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 7 16
H 7
PAULI_CHANNEL_1(0, 0, 0) 7
H 8
PAULI_CHANNEL_1(0, 0, 0) 8
CX 8 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 8 16
H 8
PAULI_CHANNEL_1(0, 0, 0) 8
H 9
PAULI_CHANNEL_1(0, 0, 0) 9
CX 9 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 9 16
H 9
PAULI

In [37]:
model = circ.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True)

In [38]:
model.diagram("matchgraph-3d")

{"accessors":[{"bufferView":0,"byteOffset":0,"componentType":5126,"count":17,"max":[0,0.400000005960464,0.400000005960464],"min":[0,-0.400000005960464,-0.400000005960464],"name":"circle_loop","type":"VEC3"},{"bufferView":1,"byteOffset":0,"componentType":5126,"count":30,"max":[12,3,6],"min":[0,0,3],"name":"buf_scattered_lines","type":"VEC3"}],"asset":{"version":"2.0"},"bufferViews":[{"buffer":0,"byteLength":204,"byteOffset":0,"name":"circle_loop","target":34962},{"buffer":1,"byteLength":360,"byteOffset":0,"name":"buf_scattered_lines","target":34962}],"buffers":[{"byteLength":204,"name":"circle_loop","uri":"data:application/octet-stream;base64,AAAAAM3MzD4AAAAAAAAAAOU1vT5Fvxw+AAAAAMPQkD7D0JA+AAAAAES/HD7lNb0+AAAAAPIwlrLNzMw+AAAAAEe/HL7lNb0+AAAAAMPQkL7D0JA+AAAAAOc1vb5Avxw+AAAAAM3MzL7yMBazAAAAAOU1vb5Evxy+AAAAAMHQkL7E0JC+AAAAADy/HL7nNb2+AAAAAPLkozHNzMy+AAAAAEm/HD7kNb2+AAAAAMbQkD6/0JC+AAAAAOY1vT5Evxy+AAAAAM3MzD4AAAAA"},{"byteLength":360,"name":"buf_scattered_lines","uri":"data:application/octet-stream;base64,AAAAAAAAAAAAAMBAAAAAAAAAQEAAAMBAAAAAAAAAAAAAAMBAAAAAAAAAQEAAAMBAAAAAAAAAAAAAAEBAAAAAAAAAQEAAAEBAAABAQAAAAAAAAMBAAABAQAAAQEAAAMBAAABAQAAAAAAAAMBAAABAQAAAQEAAAMBAAABAQAAAAAAAAEBAAABAQAAAQEAAAEBAAADAQAAAAAAAAMBAAADAQAAAQEAAAMBAAADAQAAAAAAAAMBAAADAQAAAQEAAAMBAAADAQAAAAAAAAEBAAADAQAAAQEAAAEBAAAAQQQAAAAAAAMBAAAAQQQAAQEAAAMBAAAAQQQAAAAAAAMBAAAAQQQAAQEAAAMBAAAAQQQAAAAAAAEBAAAAQQQAAQEAAAEBAAABAQQAAAAAAAMBAAABAQQAAQEAAAMBAAABAQQAAAAAAAMBAAABAQQAAQEAAAMBAAABAQQAAAAAAAEBAAABAQQAAQEAAAEBA"}],"materials":[{"doubleSided":true,"name":"black","pbrMetallicRoughness":{"baseColorFactor":[0,0,0,1],"metallicFactor":1,"roughnessFactor":1}},{"doubleSided":true,"name":"black","pbrMetallicRoughness":{"baseColorFactor":[0,0,0,1],"metallicFactor":1,"roughnessFactor":1}}],"meshes":[{"primitives":[{"attributes":{"POSITION":0},"material":0,"mode":6}]},{"primitives":[{"attributes":{"POSITION":1},"material":1,"mode":1}]}],"nodes":[{"mesh":0,"translation":[0,0,3]},{"mesh":0,"translation":[3,0,3]},{"mesh":0,"translation":[6,0,3]},{"mesh":0,"translation":[9,0,3]},{"mesh":0,"translation":[12,0,3]},{"mesh":0,"translation":[0,3,3]},{"mesh":0,"translation":[3,3,3]},{"mesh":0,"translation":[6,3,3]},{"mesh":0,"translation":[9,3,3]},{"mesh":0,"translation":[12,3,3]},{"mesh":0,"translation":[0,0,6]},{"mesh":0,"translation":[3,0,6]},{"mesh":0,"translation":[6,0,6]},{"mesh":0,"translation":[9,0,6]},{"mesh":0,"translation":[12,0,6]},{"mesh":0,"translation":[0,3,6]},{"mesh":0,"translation":[3,3,6]},{"mesh":0,"translation":[6,3,6]},{"mesh":0,"translation":[9,3,6]},{"mesh":0,"translation":[12,3,6]},{"mesh":1,"translation":[0,0,0]}],"scene":0,"scenes":[{"nodes":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}]}

In [39]:
matching = pymatching.Matching.from_detector_error_model(model)

In [40]:
sampler = circ.compile_detector_sampler()
syndrome, actual_observables = sampler.sample(shots = 1000, separate_observables=True)

In [41]:
np.shape(syndrome)

(1000, 20)

In [42]:
num_errors = 0
predicted_observables = matching.decode_batch(syndrome)
print(predicted_observables)
num_errors = np.sum(np.any(predicted_observables != actual_observables, axis=1))

print(num_errors)  # prints 8

[]
0


### Extracting Circuit-Level Tanner Graph from Detector Error Model